In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
import os
par_dir = os.path.abspath(os.path.join(os.getcwd(),os.pardir))
os.chdir(par_dir)
from multilayer_model.m2_bce import BCEcVAE
from multilayer_model.m2_mse import MSEcVAE
from multilayer_loss.l2_bce import l2_bce
from multilayer_loss.l2_mse import l2_mse
from vae_earlystopping import EarlyStopping

In [3]:
x_data = np.load('./data/metal.npy')      # Target (X)
x2_data = np.load('./data/support_norm.npy')
c_data = np.load('./data/pre_re_change_temp_logconst.npy') # Condition (C)

# 데이터 분할 (Train/Val/Test)
c_train, c_temp, x_train, x_temp, x2_train, x2_temp = train_test_split(c_data, x_data,x2_data, test_size=0.4, random_state=42)
c_val, c_test, x_val, x_test, x2_val, x2_test = train_test_split(c_temp, x_temp,x2_temp, test_size=0.5, random_state=42)

# 스케일링
c_scaler, x_scaler,x2_scaler = MinMaxScaler(), MinMaxScaler(), MinMaxScaler()
c_train = c_scaler.fit_transform(c_train)
x_train = x_scaler.fit_transform(x_train)
x2_train = x2_scaler.fit_transform(x2_train)
c_val, x_val, x2_val = c_scaler.transform(c_val), x_scaler.transform(x_val), x2_scaler.transform(x2_val)
c_test, x_test, x2_test = c_scaler.transform(c_test), x_scaler.transform(x_test), x2_scaler.transform(x2_test)

# 텐서 변환
def to_tensor(arr): return torch.tensor(arr, dtype=torch.float32)
c_train, x_train, x2_train = to_tensor(c_train), to_tensor(x_train), to_tensor(x2_train)
c_val, x_val, x2_val = to_tensor(c_val), to_tensor(x_val), to_tensor(x2_val)
c_test, x_test, x2_test = to_tensor(c_test), to_tensor(x_test), to_tensor(x2_test)

train_loader = DataLoader(TensorDataset(x_train, x2_train, c_train), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val, x2_val, c_val), batch_size=64, shuffle=False)
test_loader = DataLoader(TensorDataset(x_test, x2_test, c_test), batch_size=64, shuffle=False)

x,x2, c = next(iter(train_loader))
x_dim = x.shape[1]
x2_dim = x2.shape[1]
c_dim = c.shape[1]
z_dim = 8
z2_dim = 8
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 3. BCE 및 MSE 모델 정의
model_bce = BCEcVAE(x_dim, x2_dim, c_dim, z_dim, z2_dim).to(device)
model_mse = MSEcVAE(x_dim, x2_dim, c_dim, z_dim, z2_dim).to(device)

In [4]:
optimizer_bce = optim.Adam(model_bce.parameters(), lr=1e-3, weight_decay=1e-5)
optimizer_mse = optim.Adam(model_mse.parameters(),lr=1e-3, weight_decay = 1e-5)
es_bce= EarlyStopping(patience=40, min_delta=1e-9)
es_mse = EarlyStopping(patience=40,min_delta=1e-9)

In [5]:
for epoch in range(1, 801):
    model_bce.train()
    t_loss = 0
    for x, x2, c in train_loader:
        x, x2, c = x.to(device), x2.to(device), c.to(device)
        optimizer_bce.zero_grad()
        logit2, mu, logvar = model_bce(x, x2, c)
        loss = l2_bce(logit2, x2, mu, logvar)['loss']
        loss.backward()
        optimizer_bce.step()
        t_loss += loss.item()
    
    model_bce.eval()
    v_loss = 0
    with torch.no_grad():
        for vx, vx2, vc in val_loader:
            vx, vx2, vc = vx.to(device), vx2.to(device), vc.to(device)
            v2_logit, v_mu, v_logvar = model_bce(vx, vx2, vc)
            v_loss += l2_bce(v2_logit, vx2, v_mu, v_logvar)['loss'].item()
    
    avg_v_loss = v_loss / len(val_loader)
    if epoch % 50 == 0: print(f"Epoch {epoch} | Val Loss: {avg_v_loss:.6f}")
    if es_bce(avg_v_loss, model_bce): break

es_bce.load_best_model(model_bce)

# --- 4. MSE 모델 학습 (수치 예측) ---
print("\n[Phase 2] Training Weighted MSEcVAE...")
model_mse = MSEcVAE(x_dim, x2_dim, c_dim, z_dim,z2_dim).to(device)
optimizer_mse = optim.Adam(model_mse.parameters(), lr=1e-3, weight_decay=1e-5)
es_mse = EarlyStopping(patience=40, min_delta=1e-9)

for epoch in range(1, 801):
    model_mse.train()
    t_loss = 0
    for x, x2, c in train_loader:
        x, x2, c = x.to(device), x2.to(device), c.to(device)
        optimizer_mse.zero_grad()
        x2_hat, mu, logvar = model_mse(x, x2, c)
        loss = l2_mse(x2_hat, x2, mu, logvar)['loss']
        loss.backward()
        optimizer_mse.step()
        t_loss += loss.item()

    model_mse.eval()
    v_loss = 0
    with torch.no_grad():
        for vx, vx2, vc in val_loader:
            vx, vx2, vc = vx.to(device), vx2.to(device), vc.to(device)
            vh2, vm, vl = model_mse(vx, vx2, vc)
            v_loss += l2_mse(vh2, vx2, vm, vl)['loss'].item()
    
    avg_v_loss = v_loss / len(val_loader)
    if epoch % 50 == 0: print(f"Epoch {epoch} | Val Loss: {avg_v_loss:.6f}")
    if es_mse(avg_v_loss, model_mse): break

es_mse.load_best_model(model_mse)



EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
Epoch 50 | Val Loss: 0.306064
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 4 out of 40
EarlyStopping counter: 5 out of 40
EarlyStopping counter: 6 out of 40
EarlyStopping counter: 7 out of 40
EarlyStopping counter: 8 out of 40
EarlyStopping counter: 9 out of 40
EarlyStopping counter: 10 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1

In [10]:
import numpy as np
import torch
from sklearn.metrics import r2_score

# --- 3. 최종 평가 (Test Set) - 가장 중요한 변경 부분 ---
model_bce.eval()
model_mse.eval()
all_pred, all_true = [], []

with torch.no_grad():
    for xt, xt2, ct in test_loader:
        # A. x와 c를 통해 첫 번째 Latent z를 추출 (x2는 절대 사용 안 함)
        # BCE 모델의 인코더 활용
        h_bce = model_bce.encoder(torch.cat([xt, ct], dim=1))
        z_mu_bce = model_bce.mu_head(h_bce)
        
        # MSE 모델의 인코더 활용
        h_mse = model_mse.encoder(torch.cat([xt, ct], dim=1))
        z_mu_mse = model_mse.mu_head(h_mse)

        # B. Inference 함수 호출: 내부적으로 z2를 0으로 채워 x2를 예측
        # 정답인 xt2를 인자로 주지 않고 예측 수행
        b_logit = model_bce.inference(None, z_mu_bce) 
        b_prob = torch.sigmoid(b_logit)
        
        m_val = model_mse.inference(None, z_mu_mse)
        
        # C. 결합
        combined_s = b_prob * m_val
        
        all_pred.append(combined_s.cpu().numpy())
        all_true.append(xt2.cpu().numpy())

y_true = np.concatenate(all_true).ravel()
y_pred = np.concatenate(all_pred).ravel()
r2 = r2_score(y_true, y_pred)

In [9]:
print(r2)

-0.005482196807861328
